Notebook to Explore Text Features

The data used was created in the regressions notebook under the section dated 4/10/18 i.e. it is three dataframes of listings with average price for each

In [1]:
#starting with listing descriptions

In [2]:
import pandas as pd

In [4]:
listings = pd.read_csv('Datasources/inside_airbnb/listings.csv')

In [5]:
listings.columns

Index([u'id', u'listing_url', u'scrape_id', u'last_scraped', u'name',
       u'summary', u'space', u'description', u'experiences_offered',
       u'neighborhood_overview', u'notes', u'transit', u'access',
       u'interaction', u'house_rules', u'thumbnail_url', u'medium_url',
       u'picture_url', u'xl_picture_url', u'host_id', u'host_url',
       u'host_name', u'host_since', u'host_location', u'host_about',
       u'host_response_time', u'host_response_rate', u'host_acceptance_rate',
       u'host_is_superhost', u'host_thumbnail_url', u'host_picture_url',
       u'host_neighbourhood', u'host_listings_count',
       u'host_total_listings_count', u'host_verifications',
       u'host_has_profile_pic', u'host_identity_verified', u'street',
       u'neighbourhood', u'neighbourhood_cleansed',
       u'neighbourhood_group_cleansed', u'city', u'state', u'zipcode',
       u'market', u'smart_location', u'country_code', u'country', u'latitude',
       u'longitude', u'is_location_exact', u'prope

#Text columns:
space', u'description', u'experiences_offered',
       u'neighborhood_overview', u'notes', u'transit', u'access',
       u'interaction', u'house_rules'

In [7]:
#some basic data exploration to see what can be done

In [8]:
text_features = ['space', 'description', 'experiences_offered', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules']

In [20]:
len(listings)

6608

In [22]:
#How many rows are null
for i in text_features:
    print("Column Name: %s, Nulls Count: %i," %(i, listings[i].isnull().sum()))

Column Name: space, Nulls Count: 1635,
Column Name: description, Nulls Count: 2,
Column Name: experiences_offered, Nulls Count: 0,
Column Name: neighborhood_overview, Nulls Count: 2471,
Column Name: notes, Nulls Count: 3375,
Column Name: transit, Nulls Count: 2636,
Column Name: access, Nulls Count: 2273,
Column Name: interaction, Nulls Count: 2530,
Column Name: house_rules, Nulls Count: 1642,
